In [1]:
import re 
import os 

filename = os.path.join("./data/ewha_full_text.txt") 
f = open(filename, 'r')
text = f.read() 
pattern1 = r"제\d+장"
pattern2 = r'\[별표 \d+\] '
matches1 = re.findall(pattern1, text)
splitted = re.split(pattern1, text)
matches2 = re.findall(pattern2, splitted[-1]) 

In [2]:
print(splitted)

['이화여자대학교 학칙1946. 8. 15. 제정\n2017. 8. 16. 개정', ' 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과\n그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에\n공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.\n제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)', ' 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범\n대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각\n대학”이라 한다) (개정 2016.6.16.)\n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대\n학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보\n건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).\n(개정 2016.6.16., 2017.5.15.)\n[전문개정 2015.11.27.]제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표\n1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집\n할 수 있다. (개정 1999.2.9., 2017.5.15.)\n③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위\n로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할\n전공을 결정하게 하되 이에 필요한 사항은 총장이

In [3]:
print(matches2)

['[별표 2] ', '[별표 1] ', '[별표 1] ', '[별표 1] ', '[별표 1] ', '[별표 1] ', '[별표 1] ', '[별표 2] ', '[별표 3] ']


In [4]:
from langchain_core.documents import Document

def to_document(text: str, meta):
    if meta:
        return Document(id=meta, page_content=remove_header(text), metadata={"p_id": meta})
    else: return Document(id=meta, page_content=text, metadata={"p_id": meta})

def remove_header(text):
    return text.replace("이화여자대학교 학칙", "")


In [5]:
splits = [to_document(splitted[0], 0)] \
    + [to_document(p + chunk, i+1) for i, (p, chunk) in enumerate(zip(matches1, splitted[1:-1]))] \
    + [to_document(p + chunk, i+len(matches1)+1) for i, (p, chunk) in enumerate(zip(matches2, re.split(pattern2, splitted[-1])))]

splits

[Document(id='0', metadata={'p_id': 0}, page_content='이화여자대학교 학칙1946. 8. 15. 제정\n2017. 8. 16. 개정'),
 Document(id='1', metadata={'p_id': 1}, page_content='제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과\n그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에\n공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.\n제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)'),
 Document(id='2', metadata={'p_id': 2}, page_content='제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범\n대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각\n대학”이라 한다) (개정 2016.6.16.)\n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대\n학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보\n건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).\n(개정 2016.6.16., 2017.5.15.)\n[전문개정 2015.11.27.]제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표\n1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 

In [8]:
from langchain_engine.langchain_engine import *

load_env()
llm = get_llm(temperature=0) 

prompt_template = """
    다음은 입력된 이화여자대학교 학칙 문서입니다. 불필요한 띄어쓰기와 개행(\n) 또는 문맥상 어색한 줄바꿈을 제거하고, 올바른 문장 구조로 수정하세요. 
---
### 출력 조건:
1. 불필요한 개행(\n) 제거.
2. 문장 사이 구두점을 정확히 사용.
3. '개정'문구와 연도 사이 개행 제거. 
---
### 예제:
입력: 깊은 이론과\n그 광범하고 정밀한 응용방법 원할 때에는 「고\n등교육법 시행령」 향상을 위하여 자체평\n가를 실시한다.
출력: 깊은 이론과 그 광범하고 정밀한 응용방법은 필요할 때 사용되며, 「고등교육법 시행령」의 향상을 위하여 자체평가를 실시한다.

입력: 1학기간 15시간 이상의 수업을 1학점으\n로 한다.
출력: 1학기간 15시간 이상의 수업을 1학점으로 한다.

입력: 학점의 인정범위 등 제1항의 시행에 필요한 사항은 총장이 따로 정한다. (개정\n1998.6.23)
출력: 학점의 인정범위 등 제1항의 시행에 필요한 사항은 총장이 따로 정한다. (개정 1998.6.23)
---
### 문서:
{doc}
---
### 수정된 텍스트:
"""
chain = (
        {"doc": lambda x: x.page_content} # Use as a context
        | ChatPromptTemplate.from_template(prompt_template)
        | llm
        | StrOutputParser()
    )
new_splits_2=[]
for i, split in enumerate(splits):
    new_splits_2.append(chain.invoke(splits[i]))

In [7]:
new_splits

['이화여자대학교 학칙은 1946년 8월 15일에 제정되었으며, 2017년 8월 16일에 마지막으로 개정되었습니다.',
 '제1장 총칙\n제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에 공헌할 수 있는 지도여성을 양성함을 목적으로 한다.\n제2조(명칭) 본교는 이화여자대학교라 부른다.\n제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)',
 '제2장 편제\n제4조(대학 및 대학원)\n① 본교에는 다음 각 호의 대학을 둔다.\n1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각 대학”이라 한다) (개정 2016.6.16.)\n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다). (개정 2016.6.16., 2017.5.15.)\n[전문개정 2015.11.27.]\n제5조(학부․학과․전공 및 정원)\n① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)\n② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집할 수 있다. (개정 1999.2.9., 2017.5.15.)\n③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할 

In [10]:
new_splits_2

['이화여자대학교 학칙은 1946년 8월 15일에 제정되었으며, 2017년 8월 16일에 개정되었습니다.',
 '제1장 총칙\n제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에 공헌할 수 있는 지도여성을 양성함을 목적으로 한다.\n제2조(명칭) 본교는 이화여자대학교라 부른다.\n제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)',
 '제2장 편제\n제4조(대학 및 대학원)\n① 본교에는 다음 각 호의 대학을 둔다.\n1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각 대학”이라 한다) (개정 2016.6.16.)\n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다). (개정 2016.6.16., 2017.5.15.)\n[전문개정 2015.11.27.]\n제5조(학부․학과․전공 및 정원)\n① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)\n② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집할 수 있다. (개정 1999.2.9., 2017.5.15.)\n③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할 전공을 결정

In [11]:
new_docs = [to_document(doc, i) for i, doc in enumerate(new_splits_2)] 
new_docs

[Document(id='0', metadata={'p_id': 0}, page_content='이화여자대학교 학칙은 1946년 8월 15일에 제정되었으며, 2017년 8월 16일에 개정되었습니다.'),
 Document(id='1', metadata={'p_id': 1}, page_content='제1장 총칙\n제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에 공헌할 수 있는 지도여성을 양성함을 목적으로 한다.\n제2조(명칭) 본교는 이화여자대학교라 부른다.\n제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)'),
 Document(id='2', metadata={'p_id': 2}, page_content='제2장 편제\n제4조(대학 및 대학원)\n① 본교에는 다음 각 호의 대학을 둔다.\n1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각 대학”이라 한다) (개정 2016.6.16.)\n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다). (개정 2016.6.16., 2017.5.15.)\n[전문개정 2015.11.27.]\n제5조(학부․학과․전공 및 정원)\n① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)\n② 모집단위별 입학정원의 일부는 입학전형에 따라 2

In [12]:
with open("./data/ewha_chunk_doc_fix.json", 'w') as f:
    for doc in new_docs:
        f.write(doc.json() + '\n')